In [1]:
from math import pi, sin, cos
import json

In [2]:
file_path = '/home/afomin/projects/mj/TR_24_rc_car/tools/coordinates.json'

In [3]:
w = 0.20 # wheels base (distance)
# delta_n (position of the motor) can be in range -> 0...max_n - 
# gear_ratio = 1.0 / 73.0
gear_ratio = 1.0
wheels_radius_m = 0.065
# max_n = 65536
max_n = 20

In [4]:
def calc_delta_d(delta_n):
    # delta_n (int) - a value which represents a turn of wheel
    # return
    # delta_d (float) - the length of the path in meters
    delta_n = float(delta_n)
    return delta_n * 2 * pi * wheels_radius_m * gear_ratio / max_n

In [5]:
# can be called only when delta_d_r != delta_d_l
def calc_turning_radius(delta_d_r, delta_d_l):
    # delta_d_r (float) - meters of movement right side
    # delta_d_l (float) - meters of movement left side
    # return
    # turning radius (float) - meters
    return delta_d_r * w / (delta_d_l - delta_d_r)

In [6]:
# calc a change in a direction angle
def calc_delta_phi(delta_d_r, delta_d_l, turn_radius):
    # delta_d_r (float) - meters of movement right side
    # delta_d_l (float) - meters of movement left side
    # turn_radius (float) - meters, a turning radius
    # return
    # delta_phi (float) - rad, a diff in angle for delta
    if delta_d_l == delta_d_r:
        return 0.0
    # delta_d_l != delta_d_r
    if turn_radius == 0.0:
        return delta_d_l / w
    else:
        # turn_radius != 0.0
        return delta_d_r / turn_radius

In [7]:
# at i interval
def calc_x(delta_d_r, delta_d_l, phi_prev, phi, turn_radius):
    # delta_d_r (float) - meters of movement right side
    # delta_d_l (float) - meters of movement left side
    # phi_prev (float) - radians, at i-1 of an angle
    # phi (float) - radians, at i of an angle
    # turn_radius (float) - meters, a turning radius
    # return
    # x at interval i (float)
    if delta_d_r == delta_d_l:
        return sin(phi) * (delta_d_r + delta_d_l) / 2.0
    return (turn_radius + w / 2.0) * (
            sin(phi_prev + pi / 2.0) + sin(phi - pi / 2.0))

In [8]:
# at i interval
def calc_y(delta_d_r, delta_d_l, phi_prev, phi, turn_radius):
    # delta_d_r (float) - meters of movement right side
    # delta_d_l (float) - meters of movement left side
    # phi_prev (float) - radians, at i-1 of an angle
    # phi (float) - radians, at i of an angle
    # turn_radius (float) - meters, a turning radius
    # return
    # y at interval i (float)
    if delta_d_r == delta_d_l:
        return cos(phi) * (delta_d_r + delta_d_l) / 2.0
    return (turn_radius + w / 2.0) * (
            cos(phi_prev + pi / 2.0) + cos(phi - pi / 2.0))

In [9]:
epochs = 100
left_wheel_N = [1.1 * max_n for i in range(epochs)]
right_wheel_N = [1.0 * max_n for i in range(epochs)]


In [10]:
x_arr = []
y_arr = []
phi_arr = []
phi = 0.0
for i in range(len(left_wheel_N)):
    right_n = right_wheel_N[i]
    left_n = left_wheel_N[i]

    print(f'i: {i} r: {right_n} l: {left_n}')

    delta_d_r = calc_delta_d(right_n)
    delta_d_l = calc_delta_d(left_n)
    print(f'delta d r: {delta_d_r} l: {delta_d_l}')

    turn_radius = calc_turning_radius(
        delta_d_r,
        delta_d_l) 
    print(f'turn radius {turn_radius}')

    delta_phi = calc_delta_phi(
        delta_d_r, delta_d_l, turn_radius)
    phi_prev = phi
    phi += delta_phi
    phi_arr.append(phi)

    print(f'phi: {phi} delta phi: {delta_phi}')


    x = calc_x(delta_d_r, delta_d_l, phi_prev, phi, turn_radius)
    y = calc_y(delta_d_r, delta_d_l, phi_prev, phi, turn_radius)

    print(f'x: {x} y: {y}')

    x_arr.append(x)
    y_arr.append(y)

i: 0 r: 20.0 l: 22.0
delta d r: 0.4084070449666731 l: 0.44924774946334034
turn radius 2.000000000000005
phi: 0.20420352248333606 delta phi: 0.20420352248333606
x: 0.0436320976942918 y: 0.42585332024867645
i: 1 r: 20.0 l: 22.0
delta d r: 0.4084070449666731 l: 0.44924774946334034
turn radius 2.000000000000005
phi: 0.40840704496667213 delta phi: 0.20420352248333606
x: 0.1290831883693475 y: 0.4081572500843633
i: 2 r: 20.0 l: 22.0
delta d r: 0.4084070449666731 l: 0.44924774946334034
turn radius 2.000000000000005
phi: 0.6126105674500082 delta phi: 0.20420352248333606
x: 0.2091703073438108 y: 0.3735004589578461
i: 3 r: 20.0 l: 22.0
delta d r: 0.4084070449666731 l: 0.44924774946334034
turn radius 2.000000000000005
phi: 0.8168140899333443 delta phi: 0.20420352248333606
x: 0.2805654841423028 y: 0.32332308829408024
i: 4 r: 20.0 l: 22.0
delta d r: 0.4084070449666731 l: 0.44924774946334034
turn radius 2.000000000000005
phi: 1.0210176124166803 delta phi: 0.20420352248333606
x: 0.3403019365467533 y: 

In [11]:
with open(file_path, 'w') as f:
    json.dump(
        {
            'x': x_arr,
            'y': y_arr,
            'phi': phi_arr,
            'w': w,
        }, f, indent=4)